In [1]:
from scipy.spatial.distance import correlation
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import pairwise_distances

import json

In [2]:
id_list = list(np.load('dataset/ids.npy'))
np_arrays = np.load('dataset/features.npy')
dists = cosine_similarity(np_arrays)
# dists = pairwise_distances(np_arrays, metric='cosine')

In [27]:
dists[0]

array([ 1.        ,  0.69253287,  0.31823078, ...,  0.26890282,
        0.36782606,  0.50841996])

In [28]:
with open('dataset/user_ratings_LV_filtered.json') as fd:
    user_ratings = json.load(fd)
len(user_ratings)

183223

In [29]:
with open('dataset/business_directory_LV.json') as fd:
    business_directory = json.load(fd)
len(business_directory)

24768

In [30]:
# users = list(user_ratings.keys())
with open('dataset/user_directory_LV_filtered.json') as fd:
    users = json.load(fd)
len(users)

183223

In [31]:
# user_matrix = np.zeros((len(users), len(id_list)))
# user_matrix.shape
# for user, ratings in user_ratings.items():
#     user_idx = users.index(user)
#     for company, rating in ratings.items():
#         if (company in id_list):
#             company_idx = id_list.index(company)
#             user_matrix[user_idx][company_idx] = rating
#         else:
#             print(company)
# for i in range(len(users)):
#     temp = np.sum(user_matrix[i])
#     if (temp == 0.0):
#         print(users[i])
#         print(user_ratings[users[i]])


In [32]:
# for i in range(len(users)):
#     temp = np.sum(user_matrix[i])
#     if (temp == 0.0):
#         print(users[i])
#         print(user_ratings[users[i]])

In [33]:
# np.save('dataset/user_ratings_LV.npy', user_matrix)
user_matrix = np.load('dataset/user_ratings_LV.npy')

In [34]:
# non_zero_rating = np.nonzero(user_matrix)
zero_indication = (user_matrix != 0.0)
zero_indication

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ..., 
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]], dtype=bool)

In [35]:

non_zero_count = (zero_indication.sum(axis=1))
non_zero_count.shape

(183223,)

In [36]:
mean_user_rating = user_matrix.sum(axis=1)/non_zero_count
mean_user_rating[0]

4.0

In [37]:
ratings_diff = (user_matrix - mean_user_rating[:, np.newaxis])
ratings_diff[ratings_diff < 0] = 0

In [246]:
from sklearn.metrics.pairwise import pairwise_distances
# item_similarity = pairwise_distances(np_arrays, metric='cosine')
item_similarity = dists

In [247]:
T1 = item_similarity.dot(ratings_diff.T)
T2 = (ratings_diff.dot(item_similarity)).T
T3 = ratings_diff.dot(item_similarity)

ValueError: shapes (3923,3923) and (3069,621) not aligned: 3923 (dim 1) != 3069 (dim 0)

In [248]:
T4 = T3/np.array([np.abs(item_similarity).sum(axis=1)])

ValueError: operands could not be broadcast together with shapes (621,3069) (1,3923) 

In [249]:
# pred = mean_user_rating[:, np.newaxis] + T4
pred2 = mean_user_rating[:, np.newaxis] + T4

In [250]:
pred2.shape

(621, 3069)

In [251]:
pred[1]

NameError: name 'pred' is not defined

In [252]:
T4[0]

array([ 0.00283627,  0.00301929,  0.00292167, ...,  0.00282494,
        0.00270167,  0.00256587])

In [3]:
train_data_matrix = np.load('dataset/train_data_matrix.npy')
test_data_matrix = np.load('dataset/test_data_matrix.npy')
with (open('dataset/Sean_company_list.json')) as fd:
    company_list = json.load(fd)


In [4]:
s = set(id_list).intersection(company_list)
len(s)
extracted_np_arrays = np.zeros((len(s), np_arrays.shape[1]))
j = 0
for i in range(len(company_list)):
    company_id = company_list[i]
    
    if (company_id in id_list):
        original_idx = id_list.index(company_id)
        extracted_np_arrays[j] = np_arrays[original_idx]
        j+=1
extracted_np_arrays.shape


(3069, 387)

In [7]:
zero_indication = (train_data_matrix != 0.0)
zero_indication

array([[ True, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ..., 
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]], dtype=bool)

In [9]:
mean_user_rating = train_data_matrix.sum(axis=0)/non_zero_count
mean_user_rating[0]

/home/toriqyan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


3.5

In [22]:
mean_user_rating[np.isnan(mean_user_rating)] = 0.0

In [12]:
non_zero_count = (zero_indication.sum(axis=0))
non_zero_count.shape


(3069,)

In [18]:
sum(non_zero_count == 0)

158

In [10]:
ratings_diff = (train_data_matrix - mean_user_rating[:, np.newaxis])
ratings_diff[ratings_diff < 0] = 0

/home/toriqyan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in less
  


(621, 3069)

In [13]:
item_similarity = cosine_similarity(extracted_np_arrays)
# item_similarity= pairwise_distances(extracted_np_arrays, metric='cosine')

In [14]:
item_similarity.shape

(3069, 3069)

In [15]:
T3 = ratings_diff.dot(item_similarity)
T4 = T3/np.array([np.abs(item_similarity).sum(axis=1)])

In [17]:
pred2 = mean_user_rating[:, np.newaxis] + T4

In [26]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    # We only want to consider predicted ratings that are in the test dataset 
    # Filter out all other elements in the prediction matrix with prediction[ground_truth.nonzero()].
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [19]:
print('Item-based CF RMSE: ' + str(rmse(pred2, test_data_matrix)))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [21]:
from scipy.spatial.distance import correlation
# for i in range(len(extracted_np_arrays))
# item_similarity = correlation(extracted_np_arrays)
# ratings_diff[0].shape
zero_indication2 = (train_data_matrix > 0.0)
print(zero_indication2)
non_zero_count2 = (zero_indication2.sum(axis=1))
print(non_zero_count2.shape)
mean_user_rating2 = train_data_matrix.sum(axis=1)/non_zero_count2
print(mean_user_rating2.shape)

ratings_diff2 = (train_data_matrix - mean_user_rating2[:, np.newaxis])
# ratings_diff2[ratings_diff2 < 0] = 0

np_arrays_rating_based = ratings_diff2.T
# correlation(np_arrays_rating_based[0], np_arrays_rating_based[1])

[[ True False False ..., False False False]
 [ True False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]
(621,)
(621,)


In [22]:
# correlation(np_arrays_rating_based[0], np_arrays_rating_based[1])
# ratings_diff[0]
# zero_indication2.sum(axis=1)
# extracted_np_arrays.shape

In [23]:
# item_similarity2 = cosine_similarity(ratings_diff2)
item_similarity2 = pairwise_distances(ratings_diff2, metric='cosine')

In [24]:
item_similarity2.shape

(621, 621)

In [25]:
# T3 = ratings_diff2.dot(item_similarity2)
# T4 = T3/np.array([np.abs(item_similarity2).sum(axis=1)])

In [29]:
targets = np.transpose(test_data_matrix.nonzero())
# ratings_trans2 = ratings_diff2.T
# ratings_trans2.shape


In [28]:
prediction = []
for pos in targets:
    user = pos[0]
    item = pos[1]
    train_rating_mask = ((train_data_matrix.T)[item] > 0).astype(int)
    specific_similarity = item_similarity2[user]*train_rating_mask
    sorted_similarity_indices = np.argsort(specific_similarity)[:100]
#     print(ratings_diff2[user][sorted_similarity_indices])
#     print(specific_similarity[sorted_similarity_indices])
    temp_pred = mean_user_rating2[user]+np.sum(np_arrays_rating_based[item][sorted_similarity_indices]*specific_similarity[sorted_similarity_indices])
#     print(temp_pred)
    prediction.append(temp_pred)

In [24]:
ground_truth = test_data_matrix[test_data_matrix.nonzero()]

In [30]:
len(prediction)

5690

In [31]:
sqrt(mean_squared_error(prediction, ground_truth))

1.025337226140012

In [30]:
prediction = []
for pos in targets:
    user = pos[0]
    item = pos[1]
    prediction.append(mean_user_rating[item])

In [31]:
sqrt(mean_squared_error(prediction, ground_truth))

1.2207896034616261

(3069,)

ValueError: Found input variables with inconsistent numbers of samples: [3069, 5690]